# Data Preprocessing #

### Imports ###

In [1]:
import pandas as pd
import numpy as np
import datetime

### Data Preview ###

In [2]:
data = pd.read_csv('Animal_IO_Data_0.csv') # https://data.louisvilleky.gov/sites/default/files/Animal_IO_Data_0.csv
print("Total Observations: ", len(data))
data.head(10)

Total Observations:  150842


,AnimalID,AnimalType,IntakeDate,IntakeType,IntakeSubtype,PrimaryColor,PrimaryBreed,SecondaryBreed,Gender,SecondaryColor,...,IntakeInternalStatus,IntakeAsilomarStatus,ReproductiveStatusAtIntake,OutcomeDate,OutcomeType,OutcomeSubtype,OutcomeReason,OutcomeInternalStatus,OutcomeAsilomarStatus,ReproductiveStatusAtOutcome
0,A366370,CAT,2008-11-07 10:50:00,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,NaN,NEUTERED MALE,BROWN,...,FEARFUL,HEALTHY,ALTERED,2008-11-12 15:46:00,EUTH,FERAL,NaN,NaN,UNHEALTHY/UNTREATABLE,ALTERED
1,A366531,CAT,2008-11-10 10:20:00,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,DOMESTIC SHORTHAIR,UNKNOWN,NaN,...,NORMAL,HEALTHY,UNKNOWN,2008-11-19 20:10:00,EUTH,CONTAG DIS,NaN,SICK,HEALTHY,UNKNOWN
2,A532367,BIRD,2014-07-23 23:21:00,CONFISCATE,CRUELTY,RED,CHICKEN,NaN,MALE,BLACK,...,OTHER,HEALTHY,FERTILE,2014-11-05 15:49:00,TRANSFER,NaN,NaN,NaN,HEALTHY,FERTILE
3,A532474,OTHER,2014-07-24 18:29:00,ET REQUEST,NaN,BROWN,BAT,NaN,UNKNOWN,NaN,...,OTHER,HEALTHY,UNKNOWN,2014-07-24 23:59:00,EUTH,MEDICAL,NaN,OTHER,HEALTHY,UNKNOWN
4,A281756,DOG,2006-09-11 18:10:00,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,NaN,MALE,BROWN,...,NORMAL,HEALTHY,FERTILE,2006-09-12 13:44:00,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,FERTILE
5,A451184,BIRD,2012-01-29 15:25:00,OWNER SUR,FIELD,BLACK,CHICKEN,NaN,UNKNOWN,WHITE,...,NORMAL,HEALTHY,UNKNOWN,2012-02-22 23:59:00,TRANSFER,RESCUE GRP,NaN,NORMAL,HEALTHY,UNKNOWN
6,A256128,DOG,2005-11-26 12:35:00,STRAY,FIELD,BROWN,AMERICAN PIT BULL TERRIER,MIX,MALE,WHITE,...,NORMAL,HEALTHY,FERTILE,2005-12-08 23:59:00,EUTH,MEDICAL,NaN,NaN,HEALTHY,FERTILE
7,A314432,CAT,2007-06-15 17:13:00,OWNER SUR,OTC,BROWN TABBY,DOMESTIC SHORTHAIR,NaN,UNKNOWN,NaN,...,NORMAL,HEALTHY,UNKNOWN,2007-06-15 18:07:00,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,UNKNOWN
8,A316619,DOG,2007-06-29 20:10:00,STRAY,FIELD,WHITE,LABRADOR RETRIEVER,MIX,MALE,TAN,...,FEARFUL,HEALTHY,FERTILE,2007-07-04 13:12:00,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,FERTILE
9,A317335,CAT,2007-07-05 21:30:00,STRAY,OTC,GRAY,DOMESTIC SHORTHAIR,NaN,UNKNOWN,NaN,...,NORMAL,HEALTHY,UNKNOWN,2007-07-11 09:19:00,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,UNKNOWN


In [3]:
print(data.columns, '\n')

Index(['AnimalID', 'AnimalType', 'IntakeDate', 'IntakeType', 'IntakeSubtype',
       'PrimaryColor', 'PrimaryBreed', 'SecondaryBreed', 'Gender',
       'SecondaryColor', 'DOB', 'IntakeReason', 'IntakeInternalStatus',
       'IntakeAsilomarStatus', 'ReproductiveStatusAtIntake', 'OutcomeDate',
       'OutcomeType', 'OutcomeSubtype', 'OutcomeReason',
       'OutcomeInternalStatus', 'OutcomeAsilomarStatus',
       'ReproductiveStatusAtOutcome'],
      dtype='object') 



### Scope ###
The scope of the project was limited to only cats and dogs

In [4]:
data = data[(data['AnimalType'] == 'CAT') | (data['AnimalType'] == 'DOG')]
print("Total Observations: ", len(data))

Total Observations:  145886


### Dropping Unhelpful Rows and Columns ###

#### For the purposes of this project, OutsideAsilomarStatus is redundant (OutcomeType) and ReproductiveStatusAtOutcome is irrelevant. ####

In [5]:
data = data.drop(['OutcomeAsilomarStatus', 'ReproductiveStatusAtOutcome'], axis = 1)

#### IntakeAsilomarStatus is redundant because the same IntakeInternalStatus always has the same IntakeAsilomarStatus. ####

In [6]:
data = data.drop(['IntakeAsilomarStatus'], axis = 1)

#### Rows without an OutcomeType will not be helpful in training our model. ####

In [7]:
data = data[pd.notnull(data['OutcomeType'])]
data = data.reset_index(drop = True)

#### Certain IntakeTypes will prevent any possible intervention in the outcome and certain outcomes cannot be avoided. ####
For example, if an animal is brought in through a euthanasia request, nothing can be done to change the outcome. There are several instances of this and so they will not be included.

In [8]:
NPI_IntakeTypes = ['DISPOSAL', # disposal of dead animal
                   'ET REQUEST',  # euthanasia request
                   'EUTH REQ', # euthanasia request
                   'OUTSURGERY', # surgery
                   'QUARANTINE']  # isolated due to health/aggression reasons (almost always euthanized with fair reason)

NPI_IntakeSubtypes = ['BITE', # almost always euthanized for aggression
                      'EUTH REQ',  # euthanasia request
                      'DANGER DOG',  # dangerous dog (almost always euthanized with fair reason)
                      'POST SURG', # surgery
                      'RESCUE GRP', # always transferred
                      'POTDANGER']  # potential danger (almost always euthanized with fair reason)

NPI_OutcomeTypes = ['RTO',  # returned to owner (lost and found)
                    'DISPOSAL',  # disposing of dead animal
                    'TNR', # trap and release (animals taken and released after spayed/neutered)
                    'SNR', # similar to TNR
                    'RTF', # similar to TNR
                    'MISSING',  # uncontrollable/unknown outcome
                    'MISSING EX',  # uncontrollable/unknown outcome
                    'LOST EXP',  # uncontrollable/unknown outcome
                    'NO SHOW',  # uncontrollable/unknown outcome
                    'RELEASE',  # likely similar to TNR
                    'RELEASED']  # likely similar to TNR

NPI_ITs = data[data['IntakeType'].isin(NPI_IntakeTypes)].index
data = data.drop(NPI_ITs)

NPI_ISs = data[data['IntakeSubtype'].isin(NPI_IntakeSubtypes)].index
data = data.drop(NPI_ISs)

NPI_OTs = data[data['OutcomeType'].isin(NPI_OutcomeTypes)].index
data = data.drop(NPI_OTs)

data = data[data['IntakeInternalStatus'] != 'DEAD']  # animals that were dead on arrival
data = data[data['OutcomeSubtype'] != 'REQUESTED'] # euthanasia requested (improperly documented in IntakeType)

data = data.reset_index(drop=True)

### Dropping Columns with Many Missing Values ###

#### Checking for Missingness ####

In [9]:
data.isna().sum()[data.isna().sum() > 0]/len(data)

IntakeSubtype            0.003979
PrimaryColor             0.000046
SecondaryBreed           0.725257
SecondaryColor           0.465401
DOB                      0.416726
IntakeReason             0.765838
OutcomeDate              0.000018
OutcomeSubtype           0.049604
OutcomeReason            1.000000
OutcomeInternalStatus    0.704525
dtype: float64

#### Dropping columns with a very high percentage of missing values ####

In [10]:
data = data.drop(
    ['SecondaryBreed', 'SecondaryColor', 'IntakeReason', 'OutcomeReason', 'OutcomeInternalStatus'], 
    axis = 1)

Note: Despite having a high percentage of missing values, DOB was not dropped because of its high relevance.

### Dealing with Duplicate Entries ####

#### Checking for duplicate AnimalIDs ####

In [11]:
dupl_IDs = pd.concat(i for _, i in data.groupby("AnimalID") if len(i) > 1)
dupl_IDs = dupl_IDs.sort_values(by = ['AnimalID', 'IntakeDate'])
print("Number of duplicate AnimalIDs: ", int(len(dupl_IDs)/2))

Number of duplicate AnimalIDs:  7011


In [12]:
dupl_IDs.head(10)

,AnimalID,AnimalType,IntakeDate,IntakeType,IntakeSubtype,PrimaryColor,PrimaryBreed,Gender,DOB,IntakeInternalStatus,ReproductiveStatusAtIntake,OutcomeDate,OutcomeType,OutcomeSubtype
21436,A000000,CAT,2005-06-02 20:03:00,STRAY,FIELD,BLACK,DOMESTIC SHORTHAIR,SPAYED FEMALE,NaN,NORMAL,ALTERED,2005-06-07 08:00:00,EUTH,FERAL
3740,A000000,CAT,2006-02-02 12:27:00,STRAY,FIELD,BLACK,DOMESTIC SHORTHAIR,SPAYED FEMALE,NaN,NORMAL,ALTERED,2006-02-03 23:59:00,DIED,NaN
37717,A000000,CAT,2006-02-14 16:20:00,OWNER SUR,OTC,BLACK,DOMESTIC SHORTHAIR,SPAYED FEMALE,NaN,NORMAL,ALTERED,2006-04-20 12:00:00,EUTH,FERAL
25824,A000049,DOG,2008-11-03 19:33:00,OWNER SUR,OTC,TAN,COLLIE - ROUGH,SPAYED FEMALE,2002-03-15 00:00:00,NORMAL,ALTERED,2008-11-06 13:32:00,ADOPTION,WEB METRO
41577,A000049,DOG,2009-05-07 12:40:00,RETURN,OTC,TAN,COLLIE - ROUGH,SPAYED FEMALE,2002-03-15 00:00:00,NORMAL,ALTERED,2009-05-23 13:04:00,ADOPTION,WALK IN
65131,A000049,DOG,2012-08-04 12:22:00,OWNER SUR,OTC,TAN,COLLIE - ROUGH,SPAYED FEMALE,2002-03-15 00:00:00,NORMAL,ALTERED,2012-08-17 18:24:00,TRANSFER,NaN
9197,A000220,DOG,2005-02-11 10:07:00,STRAY,OTC,BROWN,BEAGLE,SPAYED FEMALE,2002-10-17 00:00:00,NORMAL,ALTERED,2005-03-28 11:09:00,ADOPTION,NaN
6878,A000220,DOG,2005-03-29 11:42:00,RETURN,OTC,BROWN,BEAGLE,SPAYED FEMALE,2002-10-17 00:00:00,NORMAL,ALTERED,2005-03-29 15:40:00,ADOPTION,NaN
15447,A000225,DOG,2005-03-02 10:18:00,STRAY,OTC,BLACK,PARSON (JACK) RUSSELL TERRIER,MALE,2003-02-17 00:00:00,NORMAL,FERTILE,2005-03-19 23:59:00,ADOPTION,NEW
47206,A000225,DOG,2005-03-26 10:56:00,RETURN,OTC,BLACK,PARSON (JACK) RUSSELL TERRIER,MALE,2003-02-17 00:00:00,NORMAL,FERTILE,2005-04-04 09:00:00,EUTH,BEHAV HIST


#### Dropping all but the first observation when observations are an EXACT match and reseting index ####

In [13]:
data = data.drop_duplicates()

#### Dropping latter of two observations when the same AnimalID results in 2 'EUTH'/'DIED' outcomes

In [14]:
died_IDs = dupl_IDs[(dupl_IDs['OutcomeType'] == 'EUTH') | (dupl_IDs['OutcomeType'] == 'DIED') | (dupl_IDs['OutcomeType'] == 
                                                                                                'ET PROCESS')]
dupl_died_IDs = pd.concat(i for _, i in died_IDs.groupby("AnimalID") if len(i) > 1)
dupl_TBR = dupl_died_IDs.iloc[range(1, len(dupl_died_IDs), 2)]

data = data.drop(dupl_TBR.index)
data = data.reset_index(drop = True)

#### Rechecking number of duplicate AnimalIDs ####

In [15]:
dupl_IDs = pd.concat(i for _, i in data.groupby("AnimalID") if len(i) > 1)
dupl_IDs = dupl_IDs.sort_values(by = ['AnimalID', 'IntakeDate'])
print("Number of remaining duplicate AnimalIDs: ", int(len(dupl_IDs)/2))

Number of remaining duplicate AnimalIDs:  6865


#### Removing returned animals from duplicate subset ####
Returned animals count as two observations since being returned often gave a different outcome than the initial outcome. Because of this, they should not be included in the dupl_IDs subset. Removal from this set does not have any affect on the main dataset.

In [16]:
RETURN_IDs = dupl_IDs[(dupl_IDs['IntakeType'] == 'RETURN') | (dupl_IDs['IntakeSubtype'] == 'RETURN')]['AnimalID'].values
dupl_IDs = dupl_IDs.drop(dupl_IDs[dupl_IDs['AnimalID'].isin(RETURN_IDs)].index)

print("Number of actual remaining duplicate AnimalIDs: ", int(len(dupl_IDs)/2))

Number of actual remaining duplicate AnimalIDs:  1049


#### Examining remaining duplicate AnimalIDs ####

In [17]:
dupl_IDs.head(50)

,AnimalID,AnimalType,IntakeDate,IntakeType,IntakeSubtype,PrimaryColor,PrimaryBreed,Gender,DOB,IntakeInternalStatus,ReproductiveStatusAtIntake,OutcomeDate,OutcomeType,OutcomeSubtype
21420,A000000,CAT,2005-06-02 20:03:00,STRAY,FIELD,BLACK,DOMESTIC SHORTHAIR,SPAYED FEMALE,NaN,NORMAL,ALTERED,2005-06-07 08:00:00,EUTH,FERAL
37687,A000000,CAT,2006-02-14 16:20:00,OWNER SUR,OTC,BLACK,DOMESTIC SHORTHAIR,SPAYED FEMALE,NaN,NORMAL,ALTERED,2006-04-20 12:00:00,EUTH,FERAL
2285,A000849,DOG,2005-03-23 14:43:00,OWNER SUR,FIELD,BROWN,BEAGLE,NEUTERED MALE,2002-12-18 00:00:00,NORMAL,ALTERED,2005-07-13 16:23:00,ADOPTION,NaN
21836,A000849,DOG,2005-07-22 13:05:00,OWNER SUR,OTC,BROWN,BEAGLE,NEUTERED MALE,2002-12-18 00:00:00,NORMAL,ALTERED,2005-07-26 23:59:00,ADOPTION,NaN
68782,A199523,DOG,2014-07-09 16:52:00,OWNER SUR,OTC,BLACK,BEAGLE,SPAYED FEMALE,2002-10-18 00:00:00,NORMAL,ALTERED,2014-08-21 17:45:00,ADOPTION,WALK IN
85029,A199523,DOG,2016-05-17 14:11:00,STRAY,FIELD,BLACK,BEAGLE,SPAYED FEMALE,2002-10-18 00:00:00,INJURED,ALTERED,2016-05-28 08:20:00,TRANSPORT,RESCUE GRP
7736,A238029,DOG,2005-05-09 10:53:00,STRAY,FIELD,WHITE,BASSET HOUND,SPAYED FEMALE,2003-06-03 00:00:00,NORMAL,ALTERED,2005-06-03 15:26:00,ADOPTION,NaN
51714,A238029,DOG,2010-12-31 14:29:00,OWNER SUR,FIELD,WHITE,BASSET HOUND,SPAYED FEMALE,2003-06-03 00:00:00,NORMAL,ALTERED,2011-01-16 20:28:00,TRANSFER,RESCUE GRP
40655,A239160,CAT,2005-05-18 17:32:00,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,MALE,NaN,NORMAL,FERTILE,2005-05-21 12:01:00,ADOPTION,RESCUE GRP
31464,A239160,CAT,2005-05-21 09:13:00,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,MALE,NaN,NORMAL,FERTILE,2005-05-23 10:00:00,ADOPTION,RESCUE GRP


It appears that the remaining duplicate AnimalIDs do belong do the same animals. However, these observations should still be useful because they take place under different circumstances and often have different outcomes.

### Basic Feature Modifications ###

A few feature modifications were done that will apply to both the dog and cat models.

#### Changing Genders 'NEUTERED MALE' and 'SPAYED FEMALE' to simply 'MALE' and 'FEMALE' ####
The labels 'neutered' and 'spayed' should correspond to 'fertile' and 'altered' for ReproductiveStatusAtIntake, but they do not. It was determined that due to the frequency of each value, the 'fertile' and 'altered' labels were true, whereas the 'neutered' and 'spayed' labels were incorrectly done.

In [18]:
def gender_transform(gender):

    if gender == 'NEUTERED MALE':
        return 'MALE'
    elif gender == 'SPAYED FEMALE':
        return 'FEMALE'
    else:
        return gender
    
data['Gender'] = data['Gender'].apply(gender_transform)

#### Changing OutcomeType 'ET PROCESS' to 'EUTH' ####
'ET PROCESS' is used to denote an animal that is going to be euthanized. However, this outcome type was never changed when they were euthanized.

In [19]:
def ETtoEUTH(OutcomeType):
    if OutcomeType == 'ET PROCESS':
        return 'EUTH'
    else:
        return OutcomeType
    
data['OutcomeType'] = data['OutcomeType'].apply(ETtoEUTH)

#### Changing OutcomeSubtypes 'TIME' and 'SPACE' to 'TIME/SPACE' ####
'TIME/SPACE' should cover 'TIME' and 'SPACE' and the differentiation is likely arbitrary.

In [20]:
def TStoTS(OutcomeSubtype):
    if ((OutcomeSubtype == 'TIME') | (OutcomeSubtype == 'SPACE')):
        return 'TIME/SPACE'
    else:
        return OutcomeSubtype
    
data['OutcomeSubtype'] = data['OutcomeSubtype'].apply(TStoTS)

#### Getting value from dates ####
Date columns were changed to a datetime type and were then subsequently used to creature other features, such as Intake/Outcome Month, Intake/Outcome Year, Intake/Outcome DotW (day of the week), IntakeAge, and TimeUntilOutcome.

In [21]:
IntakeDate = pd.to_datetime(data['IntakeDate'], infer_datetime_format = True).dt.strftime('%m-%d-%Y')
OutcomeDate = pd.to_datetime(data['OutcomeDate'], infer_datetime_format = True).dt.strftime('%m-%d-%Y')
DOB = pd.to_datetime(data['DOB'], infer_datetime_format = True).dt.strftime('%m-%d-%Y')

data['IntakeDate'] = pd.to_datetime(IntakeDate)
data['OutcomeDate'] = pd.to_datetime(OutcomeDate)
data['DOB'] = pd.to_datetime(DOB)

In [22]:
def get_month(date):
    return date.month

def get_year(date):
    return date.year

def get_dotw(date):
    return date.dayofweek


data['IntakeMonth'] = data['IntakeDate'].apply(get_month)
data['IntakeYear'] = data['IntakeDate'].apply(get_year)
data['IntakeDotW'] = data['IntakeDate'].apply(get_dotw)

data = data[-data['OutcomeDate'].isna()]
data['OutcomeMonth'] = data['OutcomeDate'].apply(get_month)
data['OutcomeYear'] = data['OutcomeDate'].apply(get_year)
data['OutcomeDotW'] = data['OutcomeDate'].apply(get_dotw)

In [23]:
IntakeAge = data['IntakeDate'] - data['DOB']
TimeUntilOutcome = data['OutcomeDate'] - data['IntakeDate']

In [24]:
data['IntakeAge'] = pd.to_numeric(IntakeAge.dt.days, downcast='integer')

def invalid_age(IntakeAge):
    if IntakeAge < 0:
        return np.nan
    else:
        return IntakeAge

data['IntakeAge'] = data['IntakeAge'].apply(invalid_age) # replacing negative ages with NaN

In [25]:
data['TimeUntilOutcome'] = pd.to_numeric(TimeUntilOutcome.dt.days, downcast='integer')
data = data[data['TimeUntilOutcome'] >= 0]  # removing data where OutcomeDate was before IntakeDate

#### Adding 'LiveOutcome' column ####
The model seeks to classify whether an animal is euthanized or not, so the binary 'LiveOutcome' feature is created.

In [26]:
def live_outcome(OutcomeType):
    if ((OutcomeType == 'EUTH') | (OutcomeType == 'DIED')):
        return 0
    else:
        return 1
    
data['LiveOutcome'] = data['OutcomeType'].apply(live_outcome)

#### Removing newly redundant columns ####

In [27]:
data = data.drop(['IntakeDate', 'OutcomeDate', 'DOB'], axis=1)

### Preprocessed Data Preview ###

In [28]:
print("Total Observations: ", len(data))
data.head(20)

Total Observations:  109610


,AnimalID,AnimalType,IntakeType,IntakeSubtype,PrimaryColor,PrimaryBreed,Gender,IntakeInternalStatus,ReproductiveStatusAtIntake,OutcomeType,OutcomeSubtype,IntakeMonth,IntakeYear,IntakeDotW,OutcomeMonth,OutcomeYear,OutcomeDotW,IntakeAge,TimeUntilOutcome,LiveOutcome
0,A366370,CAT,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,MALE,FEARFUL,ALTERED,EUTH,FERAL,11,2008,4,11,2008,2,NaN,5,0
1,A366531,CAT,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,UNKNOWN,NORMAL,UNKNOWN,EUTH,CONTAG DIS,11,2008,0,11,2008,2,NaN,9,0
2,A281756,DOG,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,MALE,NORMAL,FERTILE,EUTH,TIME/SPACE,9,2006,0,9,2006,1,365.0,1,0
3,A256128,DOG,STRAY,FIELD,BROWN,AMERICAN PIT BULL TERRIER,MALE,NORMAL,FERTILE,EUTH,MEDICAL,11,2005,5,12,2005,3,NaN,12,0
4,A314432,CAT,OWNER SUR,OTC,BROWN TABBY,DOMESTIC SHORTHAIR,UNKNOWN,NORMAL,UNKNOWN,EUTH,TIME/SPACE,6,2007,4,6,2007,4,NaN,0,0
5,A316619,DOG,STRAY,FIELD,WHITE,LABRADOR RETRIEVER,MALE,FEARFUL,FERTILE,EUTH,TIME/SPACE,6,2007,4,7,2007,2,NaN,5,0
6,A317335,CAT,STRAY,OTC,GRAY,DOMESTIC SHORTHAIR,UNKNOWN,NORMAL,UNKNOWN,EUTH,TIME/SPACE,7,2007,3,7,2007,2,NaN,6,0
7,A317486,CAT,STRAY,OTC,GRAY,DOMESTIC SHORTHAIR,FEMALE,NURSING,FERTILE,EUTH,TIME/SPACE,7,2007,4,7,2007,3,NaN,6,0
8,A318485,CAT,OWNER SUR,OTC,BLACK,DOMESTIC LONGHAIR,FEMALE,NORMAL,ALTERED,EUTH,TIME/SPACE,7,2007,5,7,2007,5,NaN,0,0
9,A318496,CAT,STRAY,FIELD,BLACK TIGER,DOMESTIC SHORTHAIR,UNKNOWN,NORMAL,UNKNOWN,EUTH,TIME/SPACE,7,2007,5,7,2007,3,NaN,5,0


### Saving Preprocessed Data ###

In [29]:
data.to_csv('preprocessed_data.csv')